Az adatprofilozás alapvetően az a folyamat, amelynek során az adathalmazunkat vizsgálva átfogó információkat szerzünk annak struktúrájáról, karakterisztikájáról és minőségéről. 

Alapvetően hasonló a feltáró adatelemzéshez (Exploratory Data Analysis) a vizsgálat módjában, viszont célban ezek eltérnek. Míg a feltáró adatelemzés során célunk az adatok által hordozott információk kezdetleges felfedezése és az adatokban való mintázatok észlelése az adatprofilozás során magáról az adathalmazról gyűjtünk információt, nem az adatok által hordozott jelentéstartalmat vizsgáljuk. 

Az adatprofilozás leginkább az alábbiakban segít nekünk: 

1. **Adatok felfedezése:** A meglévő adatainkat áttekinthetjük, megismerkedhetünk velük
2. **Adatok megértése:** Ide tartozik az adatok eloszlásának feltérképezése, a tipikus értékek megtalálása és egyéb leíró statisztikai mutatók elemzése
3. **Adathibák azonosítása:** Hiányző értékek, értékhibák megtalálása

### **Az adatprofilozást a "Termek" tábla egyes oszlopain fogom elvégezni**

In [1]:
SELECT COUNT(*) AS 'Sorok száma'
FROM Termek
-- 551 sor van a táblában

(1 row affected)

Total execution time: 00:00:00.105

Sorok száma
551


In [23]:
-- Mivel a listaar oszlop egy számértékű változó (informatikusosan float típusú), ezért itt van értelme a különböző leíró statisztikai mutatók értelmezésének
SELECT MIN(LISTAAR) AS 'Minimum ár', 
       MAX(LISTAAR) AS 'Maximum ár', 
       ROUND(AVG(LISTAAR),2) AS 'Átlagos ár', 
       ROUND(STDEV(LISTAAR),2) AS 'Árak szórása', 
       ROUND(VAR(LISTAAR),2) AS 'Árak varianciája'
FROM Termek
WHERE LISTAAR IS NOT NULL -- A hiányzó értékeket nem vettem bele a leíró statisztikai mutatók kiszámolásába, mivel ezekkel nem tudnának a lekérdezések mit kezdeni

(1 row affected)

Total execution time: 00:00:00.072

Minimum ár,Maximum ár,Átlagos ár,Árak szórása,Árak varianciája
0,46000,"786,28","3602,48","12977870,53"


_Levonható az a következtetés, hogy az árak alapvetően inkább jobbra elnyúló eloszlást követnek és van pár magasan kiugró érték, amelyek miatt nagy az árak szórása_

In [14]:
-- Az alábbi kóddal le tudjuk kérdezni, hogy hány hiányzó értékkel rendelkező sor van egy oszlopban, ami itt jelen esetben 527 db az 551-ből
SELECT  
    IIF(GROUPING(CASE WHEN LEIRAS IS NULL THEN 'Hiányzó érték'
         ELSE 'Van érték' 
         END) = 1, 'Összesen', CASE WHEN LEIRAS IS NULL THEN 'Hiányzó érték'
         ELSE 'Van érték' 
         END) AS 'Sor típus', 
         COUNT(*) AS 'Sorok száma'
FROM Termek
GROUP BY ROLLUP(CASE WHEN LEIRAS IS NULL THEN 'Hiányzó érték'
         ELSE 'Van érték' 
         END) 

(3 rows affected)

Total execution time: 00:00:00.031

Sor típus,Sorok száma
Hiányzó érték,527
Van érték,24
Összesen,551


_Túlnyomó többségben ebben az oszlopban hiányzó értékek vannak, ezért egy esetleges elemzés során ezzel a "LEIRAS" oszloppal nem sok mindent tudnánk kezdeni_

In [15]:
-- Feltéve, hogy az egyes termékekből a készlet minden esetben minimum 100, maximum 500 db-ra lenne meghatározva az alábbi módon tudjuk vizsgálni, hogy hány termék készlete felelne ennek meg 
DECLARE 
@min int = 100,
@max int = 500;

SELECT 
    SUM(CASE WHEN KESZLET < @min THEN 1 ELSE 0 end) AS 'Kisebb, mint az elvárt',
    SUM(CASE WHEN KESZLET BETWEEN @min and @max THEN 1 ELSE 0 end) AS 'Megfelel az elvártnak',
    SUM(CASE WHEN KESZLET > @max THEN 1 ELSE 0 end) AS 'Nagyobb, mint a megengedett'
FROM Termek

(1 row affected)

Total execution time: 00:00:00.055

"Kisebb, mint az elvárt",Megfelel az elvártnak,"Nagyobb, mint a megengedett"
6,321,224


_Ezekkel a minimum és maximum küsszöb értékekkel az 551 sorból csupán 321 felelne meg egy ilyen domain típusú megszorításnal_

In [19]:
-- Meg tudjuk nézni az alábbi kóddal a megnevezés mező karakterszám szerinti eloszlását
SELECT LEN(MEGNEVEZES) AS 'Karakterszám',
       count(*) AS 'Darabszám'
FROM Termek
GROUP BY LEN(MEGNEVEZES)
ORDER BY 'Karakterszám' ASC

(33 rows affected)

Total execution time: 00:00:00.186

Karakterszám,Darabszám
3,2
5,2
6,1
7,2
8,5
9,8
10,15
11,18
12,23
13,30


_Ezek alapján egy lapult normális eloszláshoz közelít leginkább a karakterszámok szerinti eloszlása az oszlopnak_

In [21]:
--Ki tudjuk gyűjteni például az 5 leggyakrabban előforduló értéket egy oszlopban legyen az számértékű, vagy kategórikus változó (int/float vagy varchar/nvarchar sőt dátum is lehet)
SELECT TOP 3
    MEGYS AS value, 
    COUNT(*) row_count
FROM Termek
WHERE MEGYS IS NOT null
GROUP BY MEGYS
ORDER BY COUNT(*) DESC

(3 rows affected)

Total execution time: 00:00:00.028

value,row_count
db,483
csomag,44
ív,21


_A három leggyakoribb mennyiségi egység a db, a csomag és az ív_